In [20]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import os
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
def test_model():
    key, endpoint = os.getenv('COGNITIVE_SERVICES_API_KEY'), os.getenv('COGNITIVE_SERVICES_ENDPOINT')
    model_id = os.getenv('COGNITIVE_SERVICES_MODEL_ID')
    #test_file = "./2311.07198.pdf"

    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )

    # from azure storage blob
    # file_url = "https://lgengstorage.blob.core.windows.net/training/2311.07198.pdf"
    # poller = document_analysis_client.begin_analyze_document_from_url(model_id=model_id, document_url=file_url)
    #Message: Invalid request.
    # Inner error: {
    #     "code": "InvalidManagedIdentity",
    #     "message": "The managed identity configuration is invalid: Managed identity is not enabled for the current resource."
    # }

    # from local file
    file_path = "./2311.07198.pdf"
    with open(file_path, "rb") as f: 
        poller = document_analysis_client.begin_analyze_document(model_id=model_id, document=f)

    #poller = document_analysis_client.begin_analyze_document_from_url(model_id, test_file)
    result = poller.result()

    for idx, document in enumerate(result.documents):
        print("--------Analyzing document #{}--------".format(idx + 1))
        print("Document has type {}".format(document.doc_type))
        print("Document has confidence {}".format(document.confidence))
        print("Document was analyzed by model with ID {}".format(result.model_id))
        for name, field in document.fields.items():
            field_value = field.value if field.value else field.content
            print("......"+name+"......")
            print("......found field of type '{}' with value '{}' and with confidence {}".format(field.value_type, field_value, field.confidence))


    # iterate over tables, lines, and selection marks on each page
    # for page in result.pages:
    #     print("\nLines found on page {}".format(page.page_number))
    #     for line in page.lines:
    #         print("...Line '{}'".format(line.content.encode('utf-8')))
    #     for word in page.words:
    #         print(
    #             "...Word '{}' has a confidence of {}".format(
    #                 word.content.encode('utf-8'), word.confidence
    #             )
    #         )
    #     for selection_mark in page.selection_marks:
    #         print(
    #             "...Selection mark is '{}' and has a confidence of {}".format(
    #                 selection_mark.state, selection_mark.confidence
    #             )
    #         )

    # for i, table in enumerate(result.tables):
    #     print("\nTable {} can be found on page:".format(i + 1))
    #     for region in table.bounding_regions:
    #         print("...{}".format(i + 1, region.page_number))
    #     for cell in table.cells:
    #         print(
    #             "...Cell[{}][{}] has content '{}'".format(
    #                 cell.row_index, cell.column_index, cell.content.encode('utf-8')
    #             )
    #         )
    print("-----------------------------------")


In [24]:
if __name__ == '__main__': 
    test_model()


--------Analyzing document #1--------
Document has type 20231114_2
Document has confidence 0.997
Document was analyzed by model with ID 20231114_2
......papername......
......found field of type 'string' with value 'MonoDiffusion: Self-Supervised Monocular Depth Estimation Using Diffusion Model' and with confidence 0.552
......authorlist......
......found field of type 'string' with value 'Shuwei Shao, Zhongcai Pei, Weihai Chen*, Dingchi Sun, Peter C.Y.Chen and Zhengguo Li, Fellow, IEEE' and with confidence 0.623
......abstract......
......found field of type 'string' with value 'Abstract-Over the past few years, self-supervised monocular depth estimation that does not depend on ground-truth during the training phase has received widespread attention. Most efforts focus on designing different types of network architectures and loss functions or handling edge cases, e.g., occlusion and dynamic objects. In this work, we introduce a novel self-supervised depth estimation framework, dubbed